<a href="https://colab.research.google.com/github/averma12/Beginning_Data_Science/blob/master/Factory_Pattern.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import json
import xml.etree.ElementTree as et



In [35]:
class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist

In [36]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            song_info = {
                'id': song.song_id,
                'title': song.title,
                'artist': song.artist
            }
            return json.dumps(song_info)
        elif format == 'XML':
            song_info = et.Element('song', attrib={'id': song.song_id})
            title = et.SubElement(song_info, 'title')
            title.text = song.title
            artist = et.SubElement(song_info, 'artist')
            artist.text = song.artist
            return et.tostring(song_info, encoding='unicode')
        else:
            raise ValueError(format)

In the example above, you have a basic Song class to represent a song and a SongSerializer class that can convert a song object into its string representation according to the value of the format parameter.

The .serialize() method supports two different formats: JSON and XML. Any other format specified is not supported, so a ValueError exception is raised.

In [37]:
song = Song('1', 'Water of Love', 'Dire Straits')

In [38]:
serializer = SongSerializer()

In [39]:
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [40]:
serializer.serialize(song, 'XML')

'<song id="1"><title>Water of Love</title><artist>Dire Straits</artist></song>'

**The Problems With Complex Conditional Code**

- Complex if/else statements make the code harder to read,understand and maintain.
- Any time a new format is introduced the whole function needs to be modified which can make the code hard to read and understand
- The single responsibility principle states that a module, a class, or even a method should have a single, well-defined responsibility. It should do just one thing and have only one reason to change

**Looking for a Common Interface**

- identify the common goal of each of the execution paths (or logical paths).
- The code above converts a song object to its string representation using a different format in each logical path. All have the same end goal.
- Refactor the code accordingly




**Refactor 1**
Refactor each of the if else statements into its own function according to the single responsibility principle

In [41]:
class SongSerializer:
    def serialize(self, song, format):
        if format == 'JSON':
            return self._serialize_to_json(song)
        # The rest of the code remains the same

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)
    
    
    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

**Basic Implementation of the Factory method**

The central idea in Factory Method is to provide a separate component with the responsibility to decide which concrete implementation should be used based on some specified parameter. That parameter in our example is the format.



In [42]:
class SongSerializer:
    def serialize(self, song, format):
        serializer = self._get_serializer(format)
        return serializer(song)

    def _get_serializer(self, format):
        if format == 'JSON':
            return self._serialize_to_json
        elif format == 'XML':
            return self._serialize_to_xml
        else:
            raise ValueError(format)

    def _serialize_to_json(self, song):
        payload = {
            'id': song.song_id,
            'title': song.title,
            'artist': song.artist
        }
        return json.dumps(payload)

    def _serialize_to_xml(self, song):
        song_element = et.Element('song', attrib={'id': song.song_id})
        title = et.SubElement(song_element, 'title')
        title.text = song.title
        artist = et.SubElement(song_element, 'artist')
        artist.text = song.artist
        return et.tostring(song_element, encoding='unicode')

Now suppose you want to integrate this in such a way that you are able to serialize not just a song but also other items like playlist or an album or an EP.

In [43]:
# this allows adding different tupes of properties to the serializer object
class JsonSerializer:
    def __init__(self):
        self._current_object = None

    def start_object(self, object_name, object_id):
        self._current_object = {
            'id': object_id
        }

    def add_property(self, name, value):
        self._current_object[name] = value

    def to_str(self):
        return json.dumps(self._current_object)


In [44]:
class XmlSerializer:
    def __init__(self):
        self._element = None

    def start_object(self, object_name, object_id):
        self._element = et.Element(object_name, attrib={'id': object_id})

    def add_property(self, name, value):
        prop = et.SubElement(self._element, name)
        prop.text = value

    def to_str(self):
        return et.tostring(self._element, encoding='unicode')

In [45]:

class ObjectSerializer:
    
    def __init__(self,func):
        setattr(self,func.__name__,func)
    
    def serialize(self, serializable, format):
        serializer = self.get_serializer(format)
        serializable.serialize(serializer)
        return serializer.to_str()
    
#Song is a serializable because it can be serialized. Since Object serializer is generic. Song should implement this. But since Python is a dynamic
#language and there is no type checking , Song class just needs to have this method , not necessary to implement this explicitly but it helps


The implementation of ObjectSerializer is completely generic, and it only mentions a serializable and a format as parameters.

The format is used to identify the concrete implementation of the Serializer and is resolved by the factory object. The serializable parameter refers to another abstract interface that should be implemented on any object type you want to serialize.

In [62]:
class Song:
    def __init__(self, song_id, title, artist):
        self.song_id = song_id
        self.title = title
        self.artist = artist

    def serialize(self, serializer):
        serializer.start_object('song', self.song_id)
        serializer.add_property('title', self.title)
        serializer.add_property('artist', self.artist)

In [52]:
class SerializerFactory:
    
    def __init__(self):
        self._creators = {}
        
        
    @property
    def creators(self):
        return self._creators
        
        
    #@staticmethod    
    def register_format(self, format, creator):
        self._creators[format] = creator
    
    #@staticmethod
    def get_serializer(self,format):
        if format == 'JSON':
            return JsonSerializer()
        elif format == 'XML':
            return XmlSerializer()
        else:
            raise ValueError(format)


factory = SerializerFactory()

In [0]:
del factory

In [19]:
factory.get_serializer("JSON")

In [63]:
song = Song('1', 'Water of Love', 'Dire Straits')

In [55]:
serializer = ObjectSerializer(factory.get_serializer)

In [64]:
dir(song)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'artist',
 'serialize',
 'song_id',
 'title']

In [53]:
factory =  SerializerFactory()

In [54]:
factory.get_serializer.__name__

'get_serializer'

In [56]:
serializer.get_serializer("JSON")

In [65]:
serializer.serialize(song, 'JSON')

'{"id": "1", "title": "Water of Love", "artist": "Dire Straits"}'

In [0]:
class SerializerFactory:
    
    _creators = {}
        
        
    
        
        
    @classmethod    
    def register_format(cls, format, creator):
        _creators[format] = creator
    
    @staticmethod
    def get_serializer(format):
        if format == 'JSON':
            return JsonSerializer()
        elif format == 'XML':
            return XmlSerializer()
        else:
            raise ValueError(format)


factory = SerializerFactory()

In [39]:
! pip install PyYAML

In [8]:
import yaml

In [66]:
class YamlSerializer(JsonSerializer):
    def to_str(self):
        return yaml.dump(self._current_object)


In [48]:
SerializerFactory.register_format('JSON', JsonSerializer)
SerializerFactory.register_format('XML', XmlSerializer)
SerializerFactory.register_format('YAML', YamlSerializer)


NameError: ignored